In [102]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import yfinance as yf
import time
import matplotlib.pyplot as plt
import pandas_ta as ta
from preprocessing import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
stock_symbols = [
    "RIVN",  # Rivian Automotive
    "TSLA",  # Tesla
    "NIO",   # Nio Inc ADR
    "XPEV",  # Xpeng Inc ADR
    "EVGO",  # Evgo Inc
    "LI",    # Li Auto Inc ADR
    "INDI",  # Indie Semiconductor Inc Class A
    "CHPT",  # Chargepoint Holdings Inc Class A
    "LCID",  # Lucid Group Inc
    "LAZR"   # Luminar Technologies Inc Class A
]
train_raw = yf.download(stock_symbols, start='2022-01-01', end='2024-08-31')
train_raw.head()

[*********************100%***********************]  10 of 10 completed


Price                      Adj Close                                  \
Ticker                          CHPT   EVGO   INDI   LAZR       LCID   
Date                                                                   
2022-01-03 00:00:00+00:00  19.870001  10.62  11.69  17.98  40.930000   
2022-01-04 00:00:00+00:00  19.520000  10.15  11.34  17.84  39.410000   
2022-01-05 00:00:00+00:00  18.160000   9.56  10.48  16.32  36.680000   
2022-01-06 00:00:00+00:00  17.350000   9.58  10.60  15.34  38.220001   
2022-01-07 00:00:00+00:00  16.889999   9.52  10.53  15.76  41.980000   

Price                                                                    \
Ticker                            LI        NIO        RIVN        TSLA   
Date                                                                      
2022-01-03 00:00:00+00:00  32.340000  33.470001  102.720001  399.926666   
2022-01-04 00:00:00+00:00  30.330000  31.580000  101.389999  383.196655   
2022-01-05 00:00:00+00:00  29.170000  29.879999   90.010002  362.706665   
2022-01-06 00:00:00+00:00  28.379999  29.309999   87.330002  354.899994   
2022-01-07 00:00:00+00:00  29.180000  29.299999   86.279999  342.320007   

Price                                 ...    Volume                    \
Ticker                          XPEV  ...      CHPT     EVGO     INDI   
Date                                  ...                               
2022-01-03 00:00:00+00:00  50.270000  ...   7944400  4525800   786400   
2022-01-04 00:00:00+00:00  47.599998  ...   7808100  4425800   881500   
2022-01-05 00:00:00+00:00  46.110001  ...   7696000  3808800  1287300   
2022-01-06 00:00:00+00:00  46.660000  ...  10271400  4630600   969000   
2022-01-07 00:00:00+00:00  45.049999  ...   8711900  3360700   744100   

Price                                                                       \
Ticker                        LAZR      LCID        LI       NIO      RIVN   
Date                                                                         
2022-01-03 00:00:00+00:00  5380400  45685100   7153900  64286500   8346800   
2022-01-04 00:00:00+00:00  4102900  30476900   8156400  47866800  12152200   
2022-01-05 00:00:00+00:00  6185000  43318800   6349500  42442700  18645100   
2022-01-06 00:00:00+00:00  6860400  58375500  10132000  48723200  39827100   
2022-01-07 00:00:00+00:00  4508400  78419600   8864000  40781500  17515900   

Price                                           
Ticker                          TSLA      XPEV  
Date                                            
2022-01-03 00:00:00+00:00  103931400  10873300  
2022-01-04 00:00:00+00:00  100248300  10009600  
2022-01-05 00:00:00+00:00   80119800  10182500  
2022-01-06 00:00:00+00:00   90336600   6973000  
2022-01-07 00:00:00+00:00   84164700  10044700  

[5 rows x 60 columns]

In [ ]:
start = '2022-01-01'
end = '2024-08-31'
stock_train_dic = preprocess_train(stock_symbols, start, end)
stock_train_dic['TSLA']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [204]:
data = stock_train_dic[stock_symbols[0]]
for x in stock_symbols[1:]:
    data = pd.concat([data,stock_train_dic[x]])
data.shape

(5600, 41)

In [198]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'ema_10',
       'ema_25', 'ema_50', 'ema_100', 'hma_10', 'hma_25', 'hma_50', 'hma_100',
       'macd', 'macd_h', 'macd_s', 'rsi', 'momentum', 'BBL', 'BBM', 'BBU',
       'BBB', 'BBP', 'ao', 'adx', 'dmp', 'dmn', 'chop', 'pct_change', 'target',
       'ema_rec_short', 'ema_rec_long', 'hma_rec_short', 'hma_rec_long',
       'rsi_rec', 'macd_rec', 'mom_rec', 'bbands_rec', 'ao_rec'],
      dtype='object')

In [ ]:
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
train_columns = ['ema_rec_short', 'ema_rec_long', 'hma_rec_short', 'hma_rec_long',
       'rsi_rec', 'macd_rec', 'mom_rec', 'bbands_rec', 'ao_rec', 'chop', 'adx']
train_columns2 = ['Close', 'ema_10','ema_25', 'ema_50', 'ema_100', 'hma_10', 'hma_25', 'hma_50', 'hma_100', 
                 'macd', 'macd_h', 'macd_s', 'rsi', 'momentum', 'ao', 'adx', 'dmp', 'dmn', 'chop',]
X_train, X_test, y_train, y_test = train_test_split(data[train_columns2], data['target'], test_size=.2, shuffle = True)

In [273]:

bst = XGBClassifier(n_estimators=50, max_depth=30, learning_rate=0.01, objective="binary:logistic")
bst.fit(X_train, y_train)   
preds = bst.predict(X_test)
bst.score(X_test, y_test)

0.5303571428571429

In [274]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
list(zip(y_test.to_numpy(), preds))
np.mean(np.abs(y_test.to_numpy() - preds))

0.46964285714285714

In [209]:
(((y_test.to_numpy() - 2.5)*(preds - 2.5)) > 0).mean()

1.0

In [257]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
confusion_matrix(y_test, preds)
#plt.scatter(y_test, preds)

array([[502,  97],
       [422,  99]], dtype=int64)